## Code generation

In [1]:
# import sys
# !{sys.executable} -m pip install --upgrade pip
# !{sys.executable} -m pip install sympy
# !{sys.executable} -m pip install scipy
# !{sys.executable} -m pip install matplotlib

In [2]:
# Remove matplotlib deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
# initialize sympy printing (for latex output)
from sympy import init_printing, Symbol
init_printing()

# import functions and classes for compartment models
import sys
sys.path.insert(0, "./compartor") #use local copy
from compartor import *

Here, we illustrate how to export the moment equations in LaTeX format or generate code for simulations.

We consider again the exemplary model of the paper and derive moments equations with the automated function.

In [4]:
x = Content('x')
y = Content('y')
z = Content('z')

from sympy import symbols, log, exp, sin, cos, tan
kM,h = symbols('kM h') # The Michaelis-Menten constant and hill exponent

# define intake distribution
pi_I0 = OutcomeDistribution.Poisson(Symbol("\pi_{Poiss0}(y; \lambda)"), y[0], Symbol("\lambda"))
pi_I1 = OutcomeDistribution.Poisson(Symbol("\pi_{Poiss1}(y; \lambda)"), y[1], Symbol("\lambda"))
pi_I = OutcomeDistribution.CombineIndependent(Symbol("\pi_{Poiss}(y; \lambda)"), pi_I0, pi_I1)
# define fragmentation distribution
pi_F0 = OutcomeDistribution.Uniform(Symbol("\pi_{F0}(y|x)"), y[0], 0, x[0])
pi_F1 = OutcomeDistribution.Uniform(Symbol("\pi_{F1}(y|x)"), y[1], 0, x[1])
pi_F = OutcomeDistribution.CombineIndependent(Symbol("\pi_F(y|x)"), pi_F0, pi_F1)

Birth0         = TransitionClass( [x]       -to> [x+(1,0)],       'k_b',                  name='b_0')
Death0         = TransitionClass( [x]       -to> [x+(-1,0)],       'k_d', g=x[0],    name='d_0')
Birth1         = TransitionClass( [x]       -to> [x+(0,1)],       'k_b',                  name='b_1')
Death1         = TransitionClass( [x]       -to> [x+(0,-1)],       'k_d', g=x[1],    name='d_1')

Intake        = TransitionClass( {}        -to> [y],         'k_I',         pi=pi_I, name='I')
Exit          = TransitionClass( [x]       -to> {},          'k_E',                  name='E')

Coagulation   = TransitionClass( [x] + [y] -to> [x+y],       'k_C', name='C')
Fragmentation = TransitionClass( [x]       -to> [y] + [x-y], 'k_F', g=x[0]*x[1], pi=pi_F, name='F')

transitions = [Intake, Exit, Coagulation, Fragmentation, Birth0, Death0, Birth1, Death1]
# transitions = [Fragmentation]
display_transition_classes(transitions)

                     EmptySet ---> [y],  h_I = \pi_{Poiss}(y; \lambda)*k_I                      
                               [x] ---> EmptySet,  h_E = k_E*n(x)                               
[x] + [y] ---> [x + y],  h_C = k_C*(n(y) - KroneckerDelta(x, y))*n(x)/(KroneckerDelta(x, y) + 1)
                  [x] ---> [x - y] + [y],  h_F = \pi_F(y|x)*k_F*n(x)*x[0]*x[1]                  
                            [x] ---> [(1, 0) + x],  h_b_0 = k_b*n(x)                            
                         [x] ---> [(-1, 0) + x],  h_d_0 = k_d*n(x)*x[0]                         
                            [x] ---> [(0, 1) + x],  h_b_1 = k_b*n(x)                            
                         [x] ---> [(0, -1) + x],  h_d_1 = k_d*n(x)*x[1]                         

In [5]:
M = Moment
moments = [
    M(0,0),
    M(0,0)**2,
    M(1,0),
    M(1,0)**2,
    M(0,1),
    M(0,1)**2,
    M(1,1),
]
display(moments)
equations = automated_moment_equations(2, transitions, moments, clna=True)
display_moment_equations(equations)

⎡                    2                            2                            2                    ⎤
⎣Moment(0, 0), Moment (0, 0), Moment(1, 0), Moment (1, 0), Moment(0, 1), Moment (0, 1), Moment(1, 1)⎦

Computed moment equations for desired moments $\left< N\right> $, $\left< N^{2}\right> $, $\left< M^{\left(1, 0\right)}\right> $, $\left< {\left(M^{\left(1, 0\right)}\right)}^{\mathtt{\text{2}}}\right> $, $\left< M^{\left(0, 1\right)}\right> $, $\left< {\left(M^{\left(0, 1\right)}\right)}^{\mathtt{\text{2}}}\right> $, and $\left< M^{\left(1, 1\right)}\right> $.

Equations were iteratively added for $\left< N M^{\left(1, 0\right)}\right> $, $\left< N M^{\left(0, 1\right)}\right> $, and $\left< M^{\left(0, 1\right)} M^{\left(1, 0\right)}\right> $.

                                      2                                                                                                             
d                      k_C⋅Expectation (N)   k_C⋅Expectation(N)                        k_F⋅Expectation(Moment(0, 1))⋅Expectation(Moment(1, 0))      
──(Expectation(N)) = - ─────────────────── + ────────────────── - k_E⋅Expectation(N) + ─────────────────────────────────────────────────────── + k_I
dt                              2                    2                                                      Expectation(N)                          

                                                                                                                                                             
                                                                                                                                                             
                      k_F⋅(2⋅Expectation(N⋅Moment(0, 1))⋅Expectation(Moment(1, 0)) + 2⋅Expectation(N⋅Moment(1, 0))⋅Expectation(Moment(0, 1)) + Expectation(Mo
d ⎛           ⎛ 2⎞⎞                                                                                                                                          
──⎝Expectation⎝N ⎠⎠ = ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
dt                                                                                                                                                           
                                                    

...

                                                                                                                                                          2  
d ⎛           ⎛      2      ⎞⎞          2                                                                                ⎛      2      ⎞   k_E⋅Expectation (M
──⎝Expectation⎝Moment (0, 1)⎠⎠ = \lambda ⋅k_I + 2⋅\lambda⋅k_I⋅Expectation(Moment(0, 1)) + \lambda⋅k_I - 2⋅k_E⋅Expectation⎝Moment (0, 1)⎠ + ──────────────────
dt                                                                                                                                                 Expectatio

                                                                                                                                        
oment(0, 1))                                                                                                             ⎛      2      ⎞
──────────── + k_b⋅Expectation(N) + 2⋅k_b⋅Expectation(N⋅Moment(0, 1)) + k_d⋅Expectation(Momen

                                                 2                          2                                                  2                             
d                               - k_F⋅Expectation (Moment(0, 1))⋅Expectation (Moment(1, 0)) - 2⋅(k_C + k_E + 2⋅k_d)⋅Expectation (N)⋅Expectation(Moment(0, 1))
──(Expectation(Moment(1, 1))) = ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
dt                                                                                                                                                           
                                                                                                                                                             

                               ⎛       2                                                                                                                     
⋅Expectation(Moment(1, 0)) + 2⋅⎝\lambda ⋅k_I + k_C⋅

                                                                                            2                                                                
d                                                                            k_C⋅Expectation (N)⋅Expectation(Moment(1, 0))                                   
──(Expectation(N⋅Moment(1, 0))) = \lambda⋅k_I⋅Expectation(N) + \lambda⋅k_I + ───────────────────────────────────────────── - k_C⋅Expectation(N)⋅Expectation(N
dt                                                                                                 2                                                         
                                                                                                                                                             

                                                                                                                                                             
                 k_C⋅Expectation(N⋅Moment(1, 0))   

                                                                                            2                                                                
d                                                                            k_C⋅Expectation (N)⋅Expectation(Moment(0, 1))                                   
──(Expectation(N⋅Moment(0, 1))) = \lambda⋅k_I⋅Expectation(N) + \lambda⋅k_I + ───────────────────────────────────────────── - k_C⋅Expectation(N)⋅Expectation(N
dt                                                                                                 2                                                         
                                                                                                                                                             

                                                                                                                                                             
                 k_C⋅Expectation(N⋅Moment(0, 1))   

d                                                   2                                                                                                        
──(Expectation(Moment(0, 1)⋅Moment(1, 0))) = \lambda ⋅k_I + \lambda⋅k_I⋅Expectation(Moment(0, 1)) + \lambda⋅k_I⋅Expectation(Moment(1, 0)) - 2⋅k_E⋅Expectation
dt                                                                                                                                                           

                              k_E⋅Expectation(Moment(0, 1))⋅Expectation(Moment(1, 0))                                                                        
(Moment(0, 1)⋅Moment(1, 0)) + ─────────────────────────────────────────────────────── + k_b⋅Expectation(N⋅Moment(0, 1)) + k_b⋅Expectation(N⋅Moment(1, 0)) - 2
                                                   Expectation(N)                                                                                            

                                           
⋅k_d⋅E

The LaTeX source of the ODE system can be found and copy-pasted by modifying the Math Render option of the jupyter notebook. This is done by right-clicking on the system and choosing Math Settings > Math Render > Plain Source .

From the closed equations, we can also generate code to simulate the system. Currently, Python or Julia code can be generated.

For a direct code output, the user can rely on the functions:

In [6]:
julia_code = generate_julia_code(equations, function_name="IECF2BD_tris")
print(julia_code)

# evaluate ODEs
function IECF2BD_tris_ODEs(dM, M, parameters, t)
  c3 = parameters[:\lambda] # \lambda
  c5 = parameters[:kC] # kC
  c2 = parameters[:kE] # kE
  c6 = parameters[:kF] # kF
  c1 = parameters[:kI] # kI
  c4 = parameters[:kb] # kb
  c0 = parameters[:kd] # kd
  # Number of Compartments (N)
  dM[1] = c1+1/2*c5*M[1]-c2*M[1]-1/2*c5*((M[1])^(2))+c6*M[5]*M[3]/M[1]
  # N^2
  dM[2] = (1/2*((M[1])^(2))*(2*c1+c5*((M[1])^(2))-c5*M[1]-4*c2*M[2]+2*c5*((M[1])^(3))+2*c5*M[2]+2*c2*M[1]+4*c1*M[1]-4*c5*M[2]*M[1])+c6*(M[5]*M[3]+2*M[9]*M[3]+2*M[8]*M[5])*M[1]-2*c6*M[2]*M[5]*M[3])/((M[1])^(2))
  # Total Mass of species 1
  dM[3] = c3*c1+c4*M[1]-c2*M[3]-c0*M[3]
  # M1^2
  dM[4] = c3*c1+c1*((c3)^(2))+c4*M[1]+c0*M[3]-2*c2*M[4]-2*c0*M[4]+2*c4*M[8]+c2*((M[3])^(2))/M[1]+2*c3*c1*M[3]
  # Total Mass of species 2
  dM[5] = c3*c1+c4*M[1]-c2*M[5]-c0*M[5]
  # M2^2
  dM[6] = c3*c1+c1*((c3)^(2))+c4*M[1]+c0*M[5]-2*c2*M[6]-2*c0*M[6]+2*c4*M[9]+c2*((M[5])^(2))/M[1]+2*c3*c1*M[5]
  # Sum of squared content of speci